In [1]:
import os
import sys
import Levenshtein



In [2]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('/data2/OpenLLMs/math-similarity/Bert-MLM_arXiv-MP-class_zbMath')
# model = SentenceTransformer('/data2/OpenLLMs/sentence-transformers/paraphrase-MiniLM-L6-v2')


/data1/home/jiawei/.conda/envs/open_reasonser/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute_similarity(sentences_pair):
    embeddings = model.encode(sentences_pair)
    embedding_1= model.encode(sentences_pair[0], convert_to_tensor=True)
    embedding_2 = model.encode(sentences_pair[1], convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding_1, embedding_2).cpu().numpy()[0][0]


In [26]:
embeddings.shape

(2, 384)

In [27]:
# import json
# import random

# # 读取 train.jsonl 文件
# with open('envs/MATH/dataset/train.jsonl', 'r', encoding='utf-8') as f:
#     lines = f.readlines()

# # 随机打乱行顺序
# random.shuffle(lines)

# # 选取前 500 行作为 for_gen.jsonl
# for_gen_lines = lines[:500]

# # 剩下的行作为 retain_train.jsonl
# retain_train_lines = lines[500:]

# # 将 for_gen_lines 写入 for_gen.jsonl 文件
# with open('envs/MATH/dataset/for_gen.jsonl', 'w', encoding='utf-8') as f:
#     f.writelines(for_gen_lines)

# # 将 retain_train_lines 写入 retain_train.jsonl 文件
# with open('envs/MATH/dataset/retain_train.jsonl', 'w', encoding='utf-8') as f:
#     f.writelines(retain_train_lines)

# print("文件分割完成！")

In [4]:
import re

def extract_candidate_actions(log_content):
    # 正则表达式匹配 Candidate action + 数字 的行
    pattern = re.compile(r'Candidate action \d: (.*)')
    
    # 用于存储提取的信息
    groups = []
    current_group = []
    
    for line in log_content.splitlines():
        match = pattern.match(line)
        if match:
            # 如果匹配到 Candidate action 行，提取冒号后的信息
            action_info = match.group(1)
            current_group.append(action_info)
        elif line.strip() == '********************************************************************************':
            # 如果遇到分隔符，将当前组添加到 groups 中，并重置 current_group
            if current_group:
                groups.append(current_group)
                current_group = []
    
    # 处理最后一个组
    if current_group:
        groups.append(current_group)
    
    return groups


In [5]:
def read_log_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

In [6]:
log_file_path="logs_terminal/vanila_mcts_math500.log"
log_content = read_log_file(log_file_path)

In [7]:
# 提取信息
groups = extract_candidate_actions(log_content)

# 打印结果
for i, group in enumerate(groups):
    if i > 5:
        break
    print(f"Group {i+1}:")
    for action in group:
        print(action)
    print()


Group 1:
{'action': 'We can use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right)$ to convert from rectangular coordinates to polar coordinates. ки\n', 'prob': 0.25739426139958294, 'num_token': 53, 'finish_reason': 'stop'}
{'action': 'To convert from rectangular coordinates to polar coordinates, we use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right)$. ки\n', 'prob': 0.24839519181502717, 'num_token': 52, 'finish_reason': 'stop'}
{'action': 'To convert from rectangular coordinates to polar coordinates, we use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right).$ ки\n', 'prob': 0.24785813497191633, 'num_token': 53, 'finish_reason': 'stop'}
{'action': 'To convert from rectangular coordinates to polar coordinates, we can use the formulas $r = \\sqrt{x^2 + y^2}$ and $\\theta = \\arctan \\left( \\frac{y}{x} \\right).$ ки\n', 'prob': 0.2463524118134735, 'num_token': 54, 'f

In [9]:
from Levenshtein import ratio

def calculate_levenshtein_ratios(group):
    """计算组内所有action之间的Levenshtein比率"""
    import ast
    ratios = []
    
    # 将字符串形式的字典转换为实际的字典对象
    actions = []
    for item in group:
        try:
            action_dict = ast.literal_eval(item)
            actions.append(action_dict['action'])
        except:
            continue
            
    # 计算每对action之间的比率
    for i in range(len(actions)):
        for j in range(i+1, len(actions)):
            # r = ratio(actions[i], actions[j])
            r = compute_similarity([actions[i], actions[j]])
            ratios.append(r)
            
    return ratios

# 为每个组计算比率
group_ratios_list = []
for i, group in enumerate(groups):
    if i > 5:
        break
    group_ratios = calculate_levenshtein_ratios(group)
    group_ratios_list.append(group_ratios)
    
# 计算每个组的统计信息并存储在变量中
import numpy as np
group_statistics = []

for idx, group_ratios in enumerate(group_ratios_list):
    if idx > 5:
        break
    group_stat = {}
    group_stat["组号"] = idx + 1
    if group_ratios:
        group_stat["平均相似度"] = np.mean(group_ratios)
        group_stat["最大相似度"] = np.max(group_ratios)
        group_stat["最小相似度"] = np.min(group_ratios)
        group_stat["相似度标准差"] = np.std(group_ratios)
    else:
        group_stat["无法计算相似度"] = "组内action不足"
    group_statistics.append(group_stat)

# 打印统计信息
for stat in group_statistics:
    print(stat)


{'组号': 1, '平均相似度': 0.9928252, '最大相似度': 0.9998011, '最小相似度': 0.9898933, '相似度标准差': 0.0034404902}
{'组号': 2, '平均相似度': 0.98817253, '最大相似度': 1.0000001, '最小相似度': 0.9822587, '相似度标准差': 0.008363396}
{'组号': 3, '平均相似度': 0.5680475, '最大相似度': 0.7986666, '最小相似度': 0.22002785, '相似度标准差': 0.21750563}
{'组号': 4, '平均相似度': 0.9083789, '最大相似度': 0.98200315, '最小相似度': 0.8614379, '相似度标准差': 0.052713}
{'组号': 5, '平均相似度': 0.79483134, '最大相似度': 0.9774318, '最小相似度': 0.6057078, '相似度标准差': 0.17773308}
{'组号': 6, '平均相似度': 0.9336293, '最大相似度': 0.9869937, '最小相似度': 0.8859408, '相似度标准差': 0.035924952}


In [13]:
for i in range(6):
    print(group_ratios_list[i])

[0.99027205, 0.9898933, 0.99032855, 0.9998011, 0.9938306, 0.9928255]
[1.0000001, 0.9822587, 0.9822587]
[0.31996503, 0.71736205, 0.7986666, 0.71037513, 0.22002785, 0.6418884]
[0.98200315, 0.8816957, 0.8614379]
[0.6057078, 0.9774318, 0.9728621, 0.6335184, 0.6127383, 0.9667295]
[0.9186898, 0.9709877, 0.9370198, 0.8859408, 0.9869937, 0.9021439]


In [12]:
group_statistics[:5]

[{'组号': 1,
  '平均相似度': 0.9928252,
  '最大相似度': 0.9998011,
  '最小相似度': 0.9898933,
  '相似度标准差': 0.0034404902},
 {'组号': 2,
  '平均相似度': 0.98817253,
  '最大相似度': 1.0000001,
  '最小相似度': 0.9822587,
  '相似度标准差': 0.008363396},
 {'组号': 3,
  '平均相似度': 0.5680475,
  '最大相似度': 0.7986666,
  '最小相似度': 0.22002785,
  '相似度标准差': 0.21750563},
 {'组号': 4,
  '平均相似度': 0.9083789,
  '最大相似度': 0.98200315,
  '最小相似度': 0.8614379,
  '相似度标准差': 0.052713},
 {'组号': 5,
  '平均相似度': 0.79483134,
  '最大相似度': 0.9774318,
  '最小相似度': 0.6057078,
  '相似度标准差': 0.17773308}]

In [10]:
all_ratios[:2]

[0.6230529595015577, 0.6168224299065421]